# 5.3 전이 학습

전이 학습이란, 큰 데이터셋을 확보하기 아려운 현실적인 어려움을 해결 하기 위한 학습이다.  
이미지넷 처럼 아주 큰 데이터셋을 써서 훈련된 모델의 가중치를 가져와 우리가 해결하려는 과제에 맞게 보정해서 사용하는 것을 의미한다.  
이때 아주 큰 데이터셋을 사용하여 훈련된 모델을 사전 훈련된 모델(네트워크)라고 한다.
<img src="https://thebook.io/img/080289/200.jpg" width="800" height="200"/>

전이학습을 위한 방법으로는 특성 추출과 미세 조정 기법이 있다.

# 5.3.1 특성 추출 기법

특성 추출(feature extractor)은 ImageNet 데이터셋으로 사전 훈련된 모델을 가져온 후에 마지막에 완전연결층 부분만 새로 만든다.  
즉, 학습할 때는 마지막 완결연결층(이미지 카테고리를 결정하는 부분)만 학습하고 나머지 계층들은 학습되지 않도록 한다.  

특성 추출은 이미지 분류를 위해 두 부분으로 구성된다.  
* 합성곱층: 합성곱층과 풀링층으로 구성
* 데이터 분류기(완전연결층): 추출된 특성을 입력받아 최종적으로 이미지에 대한 클래스를 분류하는 부분

사전 훈련된 네트워크의 합성곱층(가중치 고정)에 새로운 데이터를 통과시키고, 그 출력을 데이터 분류기에서 훈련시킨다.  
여기서 사용 가능한 이미지 분류 모델은 다음과 같다.  
* Xception
* Inception V3
* ResNet50
* VGG16
* VGG19
* MobilNet

<img src="https://thebook.io/img/080289/201.jpg" width="500" height="600"/>

In [2]:
# 5-12 라이브러리 호출
import os
import time
import copy
import glob
import cv2  # 앞에서 설치한 OpenCV 라이브러리
import shutil

import torch   
import torchvision # 컴퓨터 비전(computer vision)용도의 패키지
import torchvision.transforms as transforms # 데이터 전처리를 위해 사용되는 패키지
import torchvision.models as models # 다양한 파이토치 네트워크를 사용할 수 있도록 도와주는 패키지
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

예제에서 사용할 이미지 데이터에 대한 전처리 방법을 정의한다.

In [3]:
# 코드 5-13 이미지 데이터 전처리 방법 정의
data_path = '../chap05/data/catanddog/train' # 이미지 데이터가 위치한 경로 지정

transform = transforms.Compose(
    [
        transforms.Resize([256,256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]
) # No.1
train_dataset = torchvision.datasets.ImageFolder(
    data_path,
    transform = transform
) # No.2
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = 32,
    num_workers = 8,
    shuffle = True
) # No.3

print(len(train_dataset))

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../chap05/data/catanddog/train'

#### No.1

torchvision.transform은 이미지 데이터를 변환하여 모델(네트워크)의 입력으로 사용할 수 있게 변환해 줍니다. 이때 사용되는 파라미터는 다음과 같습니다.
* Resize: 이미지의 크기를 조정. 즉, 256 x 256 크기로 이미지 데이터를 조정한다.
* RandomResizedCrop: 이미지를 랜덤한 크기 및 비율로 자른다.  
* Resize와 RandomResizedCrop모두 이미지를 자르는데 사용하지만 그 용도는 다르다. Resized가 합성곱층을 통과하기 위해 이미지 크기를 조정하는 전처리 과정이라면, RandomResizedCrop은 데이터 확장 용도로 사용된다. RandomResizedCrop은 이미지를 랜덤한 비율로 자른 후 데이터 크기를 조절한다.
* RandomHorizontalFlip: 이미지를 랜덤하게 수평으로 뒤집는다.  
* ToTensor: 이미지 데이터를 텐서로 변환한다.

#### No.2

datasets.ImageFolder는 데이터로더가 데이터를 불러올 대상(혹은 경로)과 방법(혹은 전처리)을 정의하며, 사용하는 파라미터는 다음과 같다.
* 첫 번째 파라미터: 불러올 데이터가 위치한 경로
* transform: 이미지 데이터에 대한 전처리

#### No.3

데이터로더는 데이터를 불러오는 부분으로 앞에서 정의한 ImageFolder(train_dataset)을 데이터로더에 할당하는데, 이때 한 번에 불러올 데이터양을 결정하는 batch_size를 지정한다.  
또한, 추가적으로 데이터를 무작위로 섞을 것인지도 설정한다.  데이터로더에서 사용하는 파라미터는 다음과 같다.
* 첫 번째 파라미터: 데이터셋을 지정한다.  
* batch_size: 한 번에 불러올 데이터양을 결정하는 배치 크기를 설정한다.
* num_workers: 데이터를 불러올 떄 하위 프로세스를 몇 개 사용할지 설정하는데, 이때 너무 많은 하위 프로세스를 설정하게 되면 오류가 발생하거나 메모리 부족 현상이 발생할 수 있다.
* shuffle: 데이터를 무작위로 섞을지를 결정한다. True로 설정하면 데이터를 무작위로 섞어서 랜덤으로 불러온다.

다음은 train_dataset에 포함된 데이터의 개수를 출력한 결과이다.
385